# Mecanismo de Atenção e o Surgimento dos Transformers (PyTorch puro)

Este notebook demonstra o funcionamento básico do mecanismo de **atenção**, núcleo da arquitetura Transformer proposta por Vaswani et al. (2017).

Etapas:
1. Entender o cálculo da atenção escalar
2. Estender para vetores de *queries*, *keys* e *values*
3. Implementar *self-attention*
4. Visualizar pesos de atenção
5. Mostrar o efeito de múltiplas cabeças de atenção

In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

## 1. Atenção como produto escalar

A atenção mede a similaridade entre uma *query* e um conjunto de *keys*.
O peso de cada elemento é proporcional ao produto escalar entre a *query* e a *key*.

In [ ]:
query = torch.tensor([1.0, 0.0])
keys = torch.tensor([[1.0, 0.0],
                     [0.7, 0.7],
                     [0.0, 1.0]])

scores = torch.matmul(keys, query)
weights = F.softmax(scores, dim=0)

print("Scores:", scores)
print("Pesos softmax:", weights)

plt.bar(["k1", "k2", "k3"], weights.numpy())
plt.title("Pesos de Atenção (Softmax)")
plt.ylabel("Intensidade")
plt.show()

## 2. Atenção vetorial (Q, K, V)

A saída é uma média ponderada dos *values*, onde os pesos vêm da similaridade
entre *queries* e *keys*.

In [ ]:
Q = torch.tensor([[1.0, 0.0]])
K = torch.tensor([[1.0, 0.0],
                  [0.7, 0.7],
                  [0.0, 1.0]])
V = torch.tensor([[1.0, 2.0],
                  [0.5, 1.0],
                  [0.0, 3.0]])

scores = torch.matmul(Q, K.T) / np.sqrt(K.shape[1])
weights = F.softmax(scores, dim=1)
att_output = torch.matmul(weights, V)

print("Scores:", scores)
print("Pesos:", weights)
print("Saída de atenção:", att_output)

## 3. Self-Attention em uma sequência

Agora, cada posição da sequência atua como *query*, *key* e *value* simultaneamente.

In [ ]:
def self_attention(X):
    d = X.shape[-1]
    scores = torch.matmul(X, X.T) / np.sqrt(d)
    weights = F.softmax(scores, dim=1)
    out = torch.matmul(weights, X)
    return out, weights

X = torch.randn(3, 4)
out, weights = self_attention(X)

print("Pesos de atenção:\n", weights)
print("Saída:\n", out)

plt.imshow(weights.detach().numpy(), cmap="Blues")
plt.title("Mapa de Atenção (Self-Attention)")
plt.xlabel("Keys")
plt.ylabel("Queries")
plt.colorbar()
plt.show()

## 4. Multi-Head Attention

O Transformer usa múlticas cabeças de atenção para capturar diferentes padrões
de relacionamento entre tokens.

In [ ]:
class MultiHeadAttention:
    def __init__(self, d_model, num_heads):
        assert d_model % num_heads == 0
        self.d_k = d_model // num_heads
        self.num_heads = num_heads
        self.Wq = torch.randn(num_heads, d_model, self.d_k)
        self.Wk = torch.randn(num_heads, d_model, self.d_k)
        self.Wv = torch.randn(num_heads, d_model, self.d_k)

    def __call__(self, X):
        heads = []
        for i in range(self.num_heads):
            Q = X @ self.Wq[i]
            K = X @ self.Wk[i]
            V = X @ self.Wv[i]
            scores = Q @ K.T / np.sqrt(self.d_k)
            weights = F.softmax(scores, dim=-1)
            head = weights @ V
            heads.append(head)
        return torch.cat(heads, dim=-1)

X = torch.randn(5, 8)
mha = MultiHeadAttention(d_model=8, num_heads=2)
out = mha(X)
print("Saída (multi-head):", out.shape)

## 5. Comparação: média simples vs. atenção

Enquanto a média simples trata todos os tokens igualmente, a atenção aprende
a focar nos elementos mais relevantes do contexto.

In [ ]:
def simple_mean(X):
    return X.mean(dim=0)

X = torch.tensor([[1.0, 2.0],
                  [0.0, 0.0],
                  [5.0, 1.0]])

mean_vec = simple_mean(X)
att_vec, att_w = self_attention(X)

print("Média simples:", mean_vec)
print("Self-Attention (ponderada):", att_vec)
print("Pesos de atenção:\n", att_w)